In [2]:
import tensorflow as tf
import numpy as np
import os
from tensorboard.plugins import projector

# Create a log directory
log_dir = 'logs/projector'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Create some dummy data (e.g., 100 vectors of dimension 50)
embedding_data = np.random.rand(100, 50).astype(np.float32)

# Save the embeddings in a checkpoint file
embedding_var = tf.Variable(embedding_data, name='embedding')
checkpoint = tf.train.Checkpoint(embedding=embedding_var)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

# Write a metadata file (optional, but recommended)
with open(os.path.join(log_dir, 'metadata.tsv'), 'w') as f:
    for i in range(100):
        f.write(f"Label {i}\n")

# Set up the projector config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'

# Save the projector config
projector.visualize_embeddings(log_dir, config)


In [4]:
import logging
tf.get_logger().setLevel(logging.ERROR)


In [7]:
import warnings
warnings.filterwarnings(action='ignore')


# import tensorflow as tf
# from tensorflow.contrib.tensorboard.plugins import projector
# tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import gensim
import os

In [8]:


file_name = "/content/sample_data/word2vec.model"
model = gensim.models.keyedvectors.KeyedVectors.load(file_name)



In [10]:
max_size = len(model.wv.index_to_key) - 1

We learned that the dimension of word vectors will be . That is, Length of the vocabulary () Number of neurons in the hidden layer (). So, we initialize a matrix named w2v with the shape as max_size which is the vocabulary size and the model's first layer size which is the number of neurons in the hidden layer:

In [11]:
w2v = np.zeros((max_size,model.layer1_size))



Now we create a new file called metadata.tsv where we save all the words in our model and we also store the embedding of each word in the w2v matrix:


In [13]:
if not os.path.exists('projections'):
    os.makedirs('projections')

with open("projections/metadata.tsv", 'w+') as file_metadata:

    # Replace index2word with index_to_key to be compatible with Gensim 4.0.0+
    for i, word in enumerate(model.wv.index_to_key[:max_size]):

        #store the embeddings of the word
        w2v[i] = model.wv[word]

        #write the word to a file
        file_metadata.write(word + '\n')

In [15]:
with tf.device("/cpu:0"):
    embedding = tf.Variable(w2v, trainable=False, name='embedding')


In [20]:
# We no longer need tf.train.Saver in TensorFlow 2.x
# saver = tf.train.Saver()

# Use tf.train.Checkpoint to save the embedding variable
checkpoint = tf.train.Checkpoint(embedding=embedding)

# Create a directory to save the checkpoint
checkpoint_dir = 'projections/checkpoints'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Save the checkpoint
checkpoint_path = os.path.join(checkpoint_dir, "word_embeddings.ckpt")
save_path = checkpoint.save(checkpoint_path)
print(f"Checkpoint saved at: {save_path}")

# Now set up the projector config for the actual word embeddings
config = projector.ProjectorConfig()
embedding_config = config.embeddings.add()
# In TF2, the variable name might be slightly different within the checkpoint
# You might need to inspect the checkpoint contents if this path doesn't work
embedding_config.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE" # This is a common path for non-Keras variables in TF2 checkpoints
embedding_config.metadata_path = 'metadata.tsv'

# Save the projector config in the projections directory
projector_config_path = os.path.join('projections', 'projector_config.pbtxt')
with open(projector_config_path, 'w') as f:
    f.write(str(config))







Checkpoint saved at: projections/checkpoints/word_embeddings.ckpt-1


In [21]:


config = projector.ProjectorConfig()
embed= config.embeddings.add()



In [22]:
embed.tensor_name = 'embedding'
embed.metadata_path = 'metadata.tsv'

In [26]:
projector.visualize_embeddings(log_dir, config)

